## ParslDock workflow 

This notebook shows how to implement a parallel, machine learning protein docking workflow. The goal of the notebook is to show how machine learning can be used as a lightweight surrogate for computationally expensive protein docking simulations. 

Protein docking aism to predict the orientation and position of two moluecules, where one molecule is a protein, and the other is a protein or a smaller molecule (ligand). Docking is commonly used in structure-based drug design, as it can predict the strength of docking between target small molecule ligands (drugs) to target binding site (receptors). Protein docking played an important role in the race to identify COVID-19 theurapeutics. 

Docking can be computed using various simulation packages (here we use [AutoDock](https://vina.scripps.edu/)); however, these simulations can be computationally expensive and the search space of potential molecules is enormous. Therefore, given finite compute capacity, we want to carefully choose which molecules to explore. For this purpose we use machine learning to predict molecules with strong docking scores based on on previous computations (a process often called active learning). The resulting ML-in-the-loop workflow proceeds as follows.

<< INSERT FIGURE >>

We use Parsl to implement and execute the docking process in parallel. Parsl allows us to establish dependencies in the workflow and to execute the workflow on arbitrary computing infrastructure, from laptops to supercomputers. We show how Parsl's integration with Python's native concurrency library (i.e., concurrent.futures) let you write applications that dynamically respond to the completion of asynchronous tasks.

In [ ]:
import os
import subprocess
from pathlib import Path
from typing import Tuple

autodocktools_path = os.getenv('MGLTOOLS_HOME')

def smi_txt_to_pdb(smiles, pdb_file):
   
    from rdkit import Chem
    from rdkit.Chem import AllChem
    
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    # Add hydrogens to the molecule
    mol = Chem.AddHs(mol)
    # Generate a 3D conformation for the molecule
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    
    # Write the molecule to a PDB file
    writer = Chem.PDBWriter(pdb_file)
    writer.write(mol)
    writer.close()

def set_element(input_pdb_file, output_pdb_file):
    tcl_script = "set_element.tcl"
    command = (
        f"vmd -dispdev text -e {tcl_script} -args {input_pdb_file} {output_pdb_file}"
    )

    result = subprocess.check_output(command.split())
    return result


def pdb_to_pdbqt(pdb_file, pdbqt_file, ligand= True):

    script, flag = (
        ("prepare_ligand4.py", "l") if ligand else ("prepare_receptor4.py", "r")
    )

    command = (
        f"{'python2.7'}"
        f" {Path(autodocktools_path) / 'MGLToolsPckgs/AutoDockTools/Utilities24' / script}"
        f" -{flag} {pdb_file}"
        f" -o {pdbqt_file}"
        f" -U nphs_lps_waters"
    )
    result = subprocess.check_output(command.split(), encoding="utf-8")


def make_autodock_vina_config(
    input_receptor_pdbqt_file: str,
    input_ligand_pdbqt_file: str,
    output_conf_file: str,
    output_ligand_pdbqt_file: str,
    center: Tuple[float, float, float],
    size: Tuple[int, int, int],
    exhaustiveness: int = 20,
    num_modes: int = 20,
    energy_range: int = 10,
):
    
    # Format configuration file
    file_contents = (
        f"receptor = {input_receptor_pdbqt_file}\n"
        f"ligand = {input_ligand_pdbqt_file}\n"
        f"center_x = {center[0]}\n"
        f"center_y = {center[1]}\n"
        f"center_z = {center[2]}\n"
        f"size_x = {size[0]}\n"
        f"size_y = {size[1]}\n"
        f"size_z = {size[2]}\n"
        f"exhaustiveness = {exhaustiveness}\n"
        f"num_modes = {num_modes}\n"
        f"energy_range = {energy_range}\n"
        f"out = {output_ligand_pdbqt_file}\n"
    )
    # Write configuration file
    with open(output_conf_file, "w") as f:
        f.write(file_contents)


def autodock_vina(config_file, num_cpu = 8):
    autodock_vina_exe = "vina"
    try:
        command = f"{autodock_vina_exe} --config {config_file} --cpu {num_cpu}"
        result = subprocess.check_output(command.split(), encoding="utf-8")

        # find the last row of the table and extract the affinity score
        result_list = result.split('\n')
        last_row = result_list[-3]
        score = last_row.split()
        return float(score[1])
    except subprocess.CalledProcessError as e:
        print(f"Command '{e.cmd}' returned non-zero exit status {e.returncode}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
## simple test that everything is working

def docking_workflow(smiles):
    import uuid
    import os
    fname = uuid.uuid4().hex
    
    smi_txt_to_pdb(smiles, '%s.pdb' % fname)
    set_element('%s.pdb' % fname, '%s-coords.pdb' % fname) 
    pdb_to_pdbqt('%s-coords.pdb' % fname, '%s-coords.pdbqt' % fname, True)
    
    receptor = '1iep_receptor.pdbqt'
    exhaustiveness = 1
    num_cpu = 1
    #specific to 1iep receptor
    cx, cy, cz=15.614, 53.380, 15.455
    sx, sy, sz = 20, 20, 20

    make_autodock_vina_config(receptor, '%s-coords.pdbqt' % fname, '%s-config.txt' % fname, 
                              '%s-out.pdb' % fname,
                          (cx, cy, cz), (sx, sy, sz), exhaustiveness)
    
    score = autodock_vina('%s-config.txt' % fname, num_cpu)
    
    # remove files.. 
    os.remove('%s.pdb' % fname)
    os.remove('%s-coords.pdbqt' % fname)
    os.remove('%s-config.txt' % fname)
    os.remove('%s-out.pdb' % fname)

    return score

print(docking_workflow("CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C"))

# Part 1: Manual ParslDock Workflow

Before creating a parallel workflow, we first go through the steps to take a target molecule and compute the docking score against a target receptor. 

Molecules can be represented as strings using the "Simplified Molecular Input Line Entry System" format. For example, Paxalovid can be represented as "CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C".

#### 1. Convert SMILES to PDB

We first need to convert the molecule to a PDB file that can be used in the docking simulation. Protein Data Bank (PDB) format is a standard for files containing atomic coordinates. We use [RDKit](https://www.rdkit.org/), a collection of cheminformatics and machine-learning software for molecular sciences.


In [ ]:
smi_txt_to_pdb('CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C', 
               'paxalovid-molecule.pdb')

#### 2. Add coordinates

We then add coordinates to the PBD file using [VMD](https://www.ks.uiuc.edu/Research/vmd/). VMD is a molecular visualization program for displaying, animating, and analyzing large biomolecular systems using 3-D graphics and built-in scripting.


In [ ]:
set_element('paxalovid-molecule.pdb', 'paxalovid-molecule-coords.pdb') 

### 3. Convert to PDBQT

We now convert the PBD file to PDBQT format. PDBQT is a similar file format to PDB, but it a also encodes connectivity (i.e. bonds). We use AutoDockTools to do the conversion. 

In [ ]:
pdb_to_pdbqt('paxalovid-molecule-coords.pdb', 'paxalovid-molecule-coords.pdbqt', True)

#### 4. Configure Docking simulation

We create a configuration file for [AutoDock Vina](https://vina.scripps.edu/) by descrbing the target receptor and setting coordinate bounds for the docking experiment. In this case, we use the 1iep receptor.  We can set properties including the exhaustiveness, which captions the number of monte carlo simulations. 

In [ ]:
receptor = '1iep_receptor.pdbqt'
ligand = 'paxalovid-molecule-coords.pdbqt'

exhaustiveness = 1
#specific to 1iep receptor
cx, cy, cz=15.614, 53.380, 15.455
sx, sy, sz = 20, 20, 20

make_autodock_vina_config(receptor, ligand, 'paxalovid-config.txt', ligand,  (cx, cy, cz), (sx, sy, sz), exhaustiveness)

#### 5. Compute the Docking score

Finally, we use [AutoDock Vina](https://vina.scripps.edu/) to compute the docking score. We use the configuration file above and run the simulation, we take the final score produced after several rounds of simulation. 

The docking score captures the potential energy change when the protein and ligand are docked. A strong binding is represented by a negative score, weaker (or no) binders are represented by positive scores. 

In [ ]:
score = autodock_vina("paxalovid-config.txt", 1)
print(score)

## Part 2: Parallelize the workflow

When selecting drug candidates we have an enormous search space of molecules we wish to consider. We consider here a small list of 1000 orderable molecules with the aim to run the workflow across many cores concurrently. 

We use the Parsl parallel programming library to represent the workflow in Parsl. We string together the steps above so that each step will execute after the proceeding step has completed. Parsl represents each step as an asynchronous "app". When an app is called, it is intercepted by Parsl and added to a queue of tasks to execute. The application is returned a future that can be used to reference the result (note: the program will not block on that future and can continue executing waiting for the result to complete). Parsl allows us to easily parallelize across cores on a multi-core computer or across computers in the case of a cloud, cluster, or supercomputer. 

In [ ]:
import pandas as pd

smi_file_name_ligand = 'dataset_orz_original_1k.csv'

search_space = pd.read_csv(smi_file_name_ligand)
search_space = search_space[['TITLE','SMILES']]

print(search_space.head(5))

We define new versions of the functions above and annotate them as Parsl apps. To help Parsl track then flow of data between apps we add a new argument "outputs". This is used by Parsl to track the files that are produced by an app such that they can be passed to subsequent apps.

In [ ]:
from parsl import python_app, bash_app

@python_app
def parsl_smi_to_pdb(smiles, outputs=[]):
    from rdkit import Chem
    from rdkit.Chem import AllChem
    
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    
    writer = Chem.PDBWriter(outputs[0].filepath)
    writer.write(mol)
    writer.close()
    
    return True

@bash_app
def parsl_set_element(input_pdb, outputs=[]):
   
    tcl_script = "set_element.tcl"
    command = (
        f"vmd -dispdev text -e {tcl_script} -args {input_pdb} {outputs[0]}"
    )
    return command

@bash_app
def parsl_pdb_to_pdbqt(input_pdb, outputs=[], ligand = True):
    import os
    from pathlib import Path
    autodocktools_path = os.getenv('MGLTOOLS_HOME') 

    # Select the correct settings for ligand or receptor preparation
    script, flag = (
        ("prepare_ligand4.py", "l") if ligand else ("prepare_receptor4.py", "r")
    )

    command = (
        f"{'python2.7'}"
        f" {Path(autodocktools_path) / 'MGLToolsPckgs/AutoDockTools/Utilities24' / script}"
        f" -{flag} {input_pdb}"
        f" -o {outputs[0]}"
        f" -U nphs_lps_waters"
    )
    return command

@python_app
def parsl_make_autodock_config(
    input_receptor,
    input_ligand,
    output_pdbqt,
    outputs=[], 
    center=(15.614, 53.380, 15.455), size=(20, 20, 20),
    exhaustiveness=1, num_modes= 20, energy_range = 10,):
   
    # Format configuration file
    file_contents = (
        f"receptor = {input_receptor}\n"
        f"ligand = {input_ligand}\n"
        f"center_x = {center[0]}\n"
        f"center_y = {center[1]}\n"
        f"center_z = {center[2]}\n"
        f"size_x = {size[0]}\n"
        f"size_y = {size[1]}\n"
        f"size_z = {size[2]}\n"
        f"exhaustiveness = {exhaustiveness}\n"
        f"num_modes = {num_modes}\n"
        f"energy_range = {energy_range}\n"
        f"out = {output_pdbqt}\n"
        #f"log = {output_log_file}\n"
    )
    # Write configuration file
    with open(outputs[0].filepath, "w") as f:
        f.write(file_contents)
        
    return True
    
@python_app
def parsl_autodock_vina(input_config, smiles, num_cpu = 1):
    import subprocess

    autodock_vina_exe = "vina"
    try:
        command = f"{autodock_vina_exe} --config {input_config} --cpu {num_cpu}"
        #print(command)
        result = subprocess.check_output(command.split(), encoding="utf-8")

        # find the last row of the table and extract the affinity score
        result_list = result.split('\n')
        last_row = result_list[-3]
        score = last_row.split()
        return (smiles, float(score[1]))
    except subprocess.CalledProcessError as e:
        return (f"Command '{e.cmd}' returned non-zero exit status {e.returncode}")
    except Exception as e:
        return (f"Error: {e}")

@python_app
def cleanup(dock_future, pdb, pdb_coords, pdb_qt, autodoc_config, docking):
    os.remove(pdb)
    os.remove(pdb_coords)
    os.remove(pdb_qt)
    os.remove(autodoc_config)
    os.remove(docking)

#### Configure Parsl

Before running the Parsl workflow, we need to configure the compute resources to be used. Parsl has an extensible model via which different types of parallel and distributed computing resources can be used. In this case we configure Parsl to use multiple cores on the local computer (as indicated by the "max_workers=4"). We can update this configuration to use Cloud or cluster resources (e.g., via a batch scheduler).

In [ ]:
from parsl.executors import HighThroughputExecutor
from parsl.config import Config
import parsl

config = Config(
    executors=[HighThroughputExecutor(
        max_workers=4, # Allows a maximum of two workers
        cpu_affinity='block' # Prevents workers from using the same cores
    )]
)
parsl.clear()
parsl.load(config)

We now can run the same workflow as before. Note that we specify the output files to be created from each step of the workflow. You will also note that each cell returns immediately (rather than blocking as it did above). Parsl intercepts the call to each app and returns a "future". The future is a proxy for a future result. 

In [ ]:
from parsl.data_provider.files import File as PFile
smiles = 'CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C'
smi_future = parsl_smi_to_pdb(smiles,
               outputs=[PFile('parsl-pax-molecule.pdb')])

element_future = parsl_set_element(smi_future.outputs[0], outputs=[PFile('parsl-pax-molecule-coords.pdb')]) 

pdbqt_future = parsl_pdb_to_pdbqt(element_future.outputs[0], outputs=[PFile('parsl-pax-molecule-coords.pdbqt')])
 
config_future = parsl_make_autodock_config(PFile('1iep_receptor.pdbqt'), pdbqt_future.outputs[0], 
                                     'parsl-pax-molecule-out.pdb', outputs=[PFile('parsl-pax-molecule-config.txt')])

dock_future = parsl_autodock_vina(config_future.outputs[0], smiles)

Futures are unique objects as they don't yet have the result of the call. Instead we can inspect them to find out if they are done (`done()`) or we can block and wait for the  app to complete by calling `result()`.

In [ ]:
dock_future.done()

In [ ]:
dock_future.result()

Finally, as we're going to be running many simulations we will cleanup the various files that have been created. 

In [ ]:
cleanup(dock_future, smi_future.outputs[0], element_future.outputs[0], pdbqt_future.outputs[0], 
            config_future.outputs[0], PFile('parsl-pax-molecule-out.pdb'))

# Part 3: Create the ML Loop

Our next step is to create a machine learning model to estimate the outcome of new computations (i.e., docking simulations) and use it to rapidly scan the search space.

To start, let's make a function that uses our prior simulations to train a model. We are going to use RDKit and scikit-learn to train a nearest-neighbor model that uses Morgan fingerprints to define similarity. In short, the function trains a model that first populates a list of certain substructures (Morgan fingerprints, specifically) and then trains a model which predicts the docking score of a new molecule by averaging those with the most similar substructures.

Note: as we use a simple model and train on a small set of training data it is likely that the predictions are not very accurate.

First let's run a number of simulations to use to train the ML model. 

In [ ]:
from concurrent.futures import as_completed
from time import monotonic
import uuid

train_data = []
futures = []
while len(futures) < 5: 
    
    selected = search_space.sample(1).iloc[0]
    title, smiles = selected['TITLE'], selected['SMILES'] 
    
    # workflow
    fname = uuid.uuid4().hex
    
    smi_future = parsl_smi_to_pdb(smiles, outputs=[PFile('%s.pdb' % fname)])
    element_future = parsl_set_element(smi_future.outputs[0], outputs=[PFile('%s-coords.pdb'% fname)]) 
    pdbqt_future = parsl_pdb_to_pdbqt(element_future.outputs[0], outputs=[PFile('%s-coords.pdbqt' % fname)])
    config_future = parsl_make_autodock_config(PFile(receptor), pdbqt_future.outputs[0], 
                                     '%s-out.pdb' % fname, outputs=[PFile('%s-config.txt' % fname)])
    dock_future = parsl_autodock_vina(config_future.outputs[0], smiles)
    cleanup(dock_future, smi_future.outputs[0], element_future.outputs[0], pdbqt_future.outputs[0], 
            config_future.outputs[0], PFile('%s-out.pdb' % fname))

    futures.append(dock_future)


while len(futures) > 0:
    future = next(as_completed(futures))
    smiles, score = future.result()
    futures.remove(future)

    print(f'Computation for {smiles} succeeded: {score}')
    
    train_data.append({
            'smiles': smiles,
            'score': score,
            'time': monotonic()
    })
    
# print(train_data)

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from functools import partial
import numpy as np


from concurrent.futures import ProcessPoolExecutor
_pool = ProcessPoolExecutor(max_workers=1)

def compute_morgan_fingerprints(smiles: str, fingerprint_length: int, fingerprint_radius: int):
    from rdkit import Chem, DataStructs
    from rdkit.Chem import AllChem
    """Get Morgan Fingerprint of a specific SMILES string.
    Adapted from: <https://github.com/google-research/google-research/blob/
    dfac4178ccf521e8d6eae45f7b0a33a6a5b691ee/mol_dqn/chemgraph/dqn/deep_q_networks.py#L750>
    Args:
      graph (str): The molecule as a SMILES string
      fingerprint_length (int): Bit-length of fingerprint
      fingerprint_radius (int): Radius used to compute fingerprint
    Returns:
      np.array. shape = [hparams, fingerprint_length]. The Morgan fingerprint.
    """
    # Parse the molecule
    molecule = Chem.MolFromSmiles(smiles)

    # Compute the fingerprint
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(
        molecule, fingerprint_radius, fingerprint_length)
    arr = np.zeros((1,), dtype=bool)

    # ConvertToNumpyArray takes ~ 0.19 ms, while
    # np.asarray takes ~ 4.69 ms
    DataStructs.ConvertToNumpyArray(fingerprint, arr)
    return arr


class MorganFingerprintTransformer(BaseEstimator, TransformerMixin):
    """Class that converts SMILES strings to fingerprint vectors"""

    def __init__(self, length: int = 256, radius: int = 4):
        self.length = length
        self.radius = radius

    def fit(self, X, y=None):
        return self  # Do need to do anything

    def transform(self, X, y=None):
        """Compute the fingerprints
        
        Args:
            X: List of SMILES strings
        Returns:
            Array of fingerprints
        """
        
        fps = []
        for x in X: 
            fps.append(compute_morgan_fingerprints(x, self.length, self.radius))
            
        return fps     

def train_model(training_data):
    """Train a machine learning model using Morgan Fingerprints.
    
    Args:
        train_data: Dataframe with a 'smiles' and 'score' column
            that contains molecule structure and docking score, respectfully.
    Returns:
        A trained model
    """
    
    # Imports for python functions run remotely must be defined inside the function
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.pipeline import Pipeline
    
    
    model = Pipeline([
        ('fingerprint', MorganFingerprintTransformer()),
        ('knn', KNeighborsRegressor(n_neighbors=4, weights='distance', metric='jaccard', n_jobs=-1))  # n_jobs = -1 lets the model run all available processors
    ])
    
    return model.fit(training_data['smiles'], training_data['score'])

def run_model(model, smiles):
    """Run a model on a list of smiles strings
    
    Args:
        model: Trained model that takes SMILES strings as inputs
        smiles: List of molecules to evaluate
    Returns:
        A dataframe with the molecules and their predicted outputs
    """
    import pandas as pd
    pred_y = model.predict(smiles)
    return pd.DataFrame({'smiles': smiles, 'score': pred_y})

Now let's train the model and run simulations over the remaining data

In [ ]:
training_df = pd.DataFrame(train_data)
m = train_model(training_df)
predictions = run_model(m, search_space['SMILES'])
predictions.sort_values('score', ascending=True).head(5)

# Part 4: Putting it all together

We now combine the parallel ParslDock workflow with the machine learning algorithm in an iterative fashion. Here each round will 1) train a machine learning model based on previous simulations; 2) apply the machine learning model to all remaining molecules; 3) select the top predicted scores; 4) run simulations on the top molecules. 

In [ ]:
futures = []
train_data = []
smiles_simulated = []
initial_count = 5
num_loops = 3
batch_size = 3

# start with an initial set of random smiles
for i in range(initial_count):
    selected = search_space.sample(1).iloc[0]
    title, smiles = selected['TITLE'], selected['SMILES'] 

    # workflow
    fname = uuid.uuid4().hex
    
    smi_future = parsl_smi_to_pdb(smiles, outputs=[PFile('%s.pdb' % fname)])
    element_future = parsl_set_element(smi_future.outputs[0], outputs=[PFile('%s-coords.pdb'% fname)]) 
    pdbqt_future = parsl_pdb_to_pdbqt(element_future.outputs[0], outputs=[PFile('%s-coords.pdbqt' % fname)])
    config_future = parsl_make_autodock_config(PFile(receptor), pdbqt_future.outputs[0], 
                                     '%s-out.pdb' % fname, outputs=[PFile('%s-config.txt' % fname)])
    dock_future = parsl_autodock_vina(config_future.outputs[0], smiles)
    cleanup(dock_future, smi_future.outputs[0], element_future.outputs[0], pdbqt_future.outputs[0], 
            config_future.outputs[0], PFile('%s-out.pdb' % fname))

    futures.append(dock_future)

# wait for all the futures to finish
while len(futures) > 0:
    future = next(as_completed(futures))
    smiles, score = future.result()
    futures.remove(future)

    print(f'Computation for {smiles} succeeded: {score}')
    
    train_data.append({
            'smiles': smiles,
            'score': score,
            'time': monotonic()
    })
    smiles_simulated.append(smiles)
   
  
# train model, run inference, and run more simulations
for i in range(num_loops):
    print(f"\nStarting batch {i}")
    m = train_model(training_df)
    predictions = run_model(m, search_space['SMILES'])
    predictions.sort_values('score', ascending=True, inplace=True) #.head(5)
    
    train_data = [] 
    futures = []
    batch_count = 0
    for smiles in predictions['smiles']:
        if smiles not in smiles_simulated:
            fname = uuid.uuid4().hex

            smi_future = parsl_smi_to_pdb(smiles, outputs=[PFile('%s.pdb' % fname)])
            element_future = parsl_set_element(smi_future.outputs[0], outputs=[PFile('%s-coords.pdb'% fname)]) 
            pdbqt_future = parsl_pdb_to_pdbqt(element_future.outputs[0], outputs=[PFile('%s-coords.pdbqt' % fname)])
            config_future = parsl_make_autodock_config(PFile(receptor), pdbqt_future.outputs[0], 
                                             '%s-out.pdb' % fname, outputs=[PFile('%s-config.txt' % fname)])
            dock_future = parsl_autodock_vina(config_future.outputs[0], smiles)
            cleanup(dock_future, smi_future.outputs[0], element_future.outputs[0], pdbqt_future.outputs[0], 
                    config_future.outputs[0], PFile('%s-out.pdb' % fname))

            futures.append(dock_future)
            
            batch_count += 1
            
        if batch_count > batch_size: 
            break

    # wait for all the workflows to complete
    while len(futures) > 0:
        future = next(as_completed(futures))
        smiles, score = future.result()
        futures.remove(future)

        print(f'Computation for {smiles} succeeded: {score}')

        train_data.append({
                'smiles': smiles,
                'score': score,
                'time': monotonic()
        })
        smiles_simulated.append(smiles)
   
                     
    training_df = pd.concat((training_df, pd.DataFrame(train_data)), ignore_index=True)

## Plotting progress

We can plot our simulations over time. We see in the plot below the docking score (y-axis) vs application time (x-axis). We show a dashed line of the "best" docking score discovered to date. You should see a step function improving the best candidate over each iteration. You should also see that the individual points tend to get lower over time.

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(figsize=(4.5, 3.))

ax.scatter(training_df['time'], training_df['score'])
ax.step(training_df['time'], training_df['score'].cummin(), 'k--')

ax.set_xlabel('Walltime (s)')
ax.set_ylabel('Docking Score)')

fig.tight_layout()